In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('../api_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

In [12]:
#Podríamos establecer que tuviera memoria
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history") #ponemos una denominada clave a la memoria "chat_history"

In [14]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

funcion_embedding = OpenAIEmbeddings(openai_api_key=api_key)
persist_path="./ejemplosk_embedding_db"
vector_store_connection = SKLearnVectorStore(embedding=funcion_embedding, persist_path=persist_path, serializer="parquet")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vector_store_connection.as_retriever())

In [15]:
from langchain.agents import tool

In [16]:
@tool
def consulta_interna(text: str) -> str:
    '''Retorna respuestas sobre la historia de España. Se espera que la entrada sea una cadena de texto
    y retorna una cadena con el resultado más relevante. Si la respuesta con esta herramienta es relevante,
    no debes usar ninguna herramienta más ni tu propio conocimiento como LLM'''
    compressed_docs = compression_retriever.invoke(text)
    resultado = compressed_docs[0].page_content
    return resultado

In [17]:
tools = load_tools(["wikipedia","llm-math"],llm=llm)

In [18]:
tools=tools+[consulta_interna]

In [19]:
agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,memory=memory,verbose=True)

In [20]:
agent.invoke("¿Qué periodo abarca cronológicamente en España el siglo de oro?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: consulta_interna
Action Input: ¿Qué periodo abarca cronológicamente en España el siglo de oro?
Observation: El Siglo de Oro español fue un período floreciente en las artes y las letras hispanas que abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.
Thought:Do I need to use a tool? No
AI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.

> Finished chain.


{'input': '¿Qué periodo abarca cronológicamente en España el siglo de oro?',
 'chat_history': '',
 'output': 'El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.'}

In [21]:
agent.invoke("¿Qué pasó durante la misma etapa en Francia?") #Gracias a tener memoria compara en esas fechas qué ocurrió en Francia



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: consulta_interna
Action Input: El Siglo de Oro en Francia
Observation: El Siglo de Oro español fue un período floreciente en las artes y las letras hispanas que abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.
Thought:Do I need to use a tool? No
AI: Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, las artes y la cultura en Francia, con figuras destacadas como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne, y otros.

> Finished chain.


{'input': '¿Qué pasó durante la misma etapa en Francia?',
 'chat_history': 'Human: ¿Qué periodo abarca cronológicamente en España el siglo de oro?\nAI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.',
 'output': 'Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, las artes y la cultura en Francia, con figuras destacadas como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne, y otros.'}

In [22]:
agent.invoke("¿Cuáles son las marcas de vehículos más famosas hoy en día?") #Pregunta que no podemos responder con nuestra BD Vectorial



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: List of famous vehicle brands
Observation: Page: List of sports cars
Summary: This page is a compilation of sports cars, coupés, roadsters, kit cars, supercars, hypercars, electric sports cars, race cars, and super SUVs, both discontinued and still in production (or will be planned to produce).  Cars that have sport trims (such as the Honda Civic SI) will be listed under the sport trims section.  Production tunes will include cars modified by outside brands and then sold. This does not include in-house brands such as Ford's Special Vehicle Team, which will be included in the main list.  Some vehicles are sold under different brands, therefore some vehicles may be listed more than once but usually link to the same page.  Different countries/continents may also classify vehicles differently.



Page: Automobile manufacturers and brands of China
Summary: There are currently about 

{'input': '¿Cuáles son las marcas de vehículos más famosas hoy en día?',
 'chat_history': 'Human: ¿Qué periodo abarca cronológicamente en España el siglo de oro?\nAI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.\nHuman: ¿Qué pasó durante la misma etapa en Francia?\nAI: Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, las artes y la cultura en Francia, con figuras destacadas como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne, y otros.',
 'output': 'Some of the most famous vehicle brands today include Toyota, Volkswagen, Ford, Chevrolet, Honda, Nissan, BMW, Mercedes-Benz, and Audi, among others. These brands are well-known for producing a wide range of vehicles, from compact cars to luxury SUVs and sp